In [14]:
"""
@author: Aleph Aseffa
Date created: 7/14/2019

Contains all the functions needed for the initialization of the game.


Functions that will be included:
- print header
- ask how many players will be playing
- for each one, take their username

"""

def username_form():
  player_name1 = input("Player 1, please enter a username:")
  player_name2 = input("Player 2, please enter a username:")
  return player_name1, player_name2

# n1, n2 = username_form()
# print(n1 + '.' + n2)


Player 1, please enter a username:Ryan
Player 2, please enter a username:Lucy
Ryan.Lucy


In [3]:
"""
@author: Aleph Aseffa
"""


class Card:
    def __init__(self, card_name, color_group, card_cost, house_cost, houses_built, rent_prices, mortgage_amt, owner, mortgaged):
        self.card_name = card_name                  # str
        self.color_group = color_group              # str
        self.card_cost = card_cost                  # int
        self.house_cost = house_cost                # int
        self.houses_built = houses_built            # int
        self.rent_prices = rent_prices              # int
        self.mortgage_amt = mortgage_amt            # int
        self.owner = owner                          # str
        self.mortgaged = mortgaged                  # bool

    def mortgage(self, player):
        """
        Sets the card's mortgaged status to True and updates the player's balance.
        :param player: An instance of the Player class.
        :return: None.
        """
        player.add_balance(self.mortgage_amt)
        self.mortgaged = True

    def sell(self, player):
        """
        Returns ownership of the card to the Bank and updates the player's balance.
        :param player: An instance of the Player class.
        :return: None.
        """
        player.add_balance(self.card_cost)
        self.owner = 'Bank'

    def purchase_card(self, player):
        """
        Gives ownership of the card to the Bank and updates the player's balance.
        :param player: An instance of the Player class.
        :return: None.
        """
        if self.card_cost > player.balance:
            print("You cannot afford this card at the moment.")
        else:
            player.cards_owned.append(self)
            player.reduce_balance(self.card_cost)
            self.owner = player

    def construct_house(self, player):
        """
        Updates number of houses that have been built on the card.
        :param player: An instance of the Player class.
        :return: None.
        """
        if self.house_cost > player.balance:
            print("You cannot afford a house on this property at the moment.")
        elif self.houses_built == 5:
            print("You have built the maximum number of houses on this property.")
        else:
            self.houses_built += 1
            print(f"You have built a house on {self.card_name}.")


def locate_card_object(name, board):

    for card in board:
        if card.card_name == name:
            card_object = card
            break

    return card_object


In [ ]:
"""
Contains the code for the AI that can play against the user.
"""


def move(ai, board):

    if ai.in_jail:
        leave_jail(ai, board)

    dice_roll = ai.roll_dice()
    ai.move_player(dice_roll)
    curr_card = board[ai.current_pos % 40]
    purchasable = ai.check_pos(board)

    if purchasable and ai.balance > curr_card.card_cost:
        curr_card.purchase_card(ai)
        print(f"{ai.name} has purchased {curr_card.card_name}")


def leave_jail(ai, board):
    ai.reduce_balance(50)
    ai.in_jail = False
    print(f"{ai.name} has paid the $50 bail and has been released from jail.")
    move(ai, board)


# TODO: evaluate what the human gains from the trade and factor that into the cost/benefit analysis.
def evaluate_trade(human, ai, money_offered, properties_offered, money_wanted, properties_wanted, board):
    personal_gain = 0
    human_gain = 0

    properties_valuation_personal = 0

    benefit = 0

    if ai.balance < 200:
        multiplier = 5
    else:
        multiplier = 2.5

    # evaluate the value of the properties to be exchanged:
    if properties_offered[0] == '':
        pass
    else:
        for card_name in properties_offered:
            card_object = locate_card_object(card_name, board)
            personal_gain += card_object.card_cost

    if properties_wanted[0] == '':
        pass
    else:
        for card_name in properties_wanted:
            card_object = locate_card_object(card_name, board)
            human_gain += card_object.card_cost

        # how much does the AI care about these properties?
        for card_name in properties_offered:

            card_offered = locate_card_object(card_name, board)

            properties_valuation_personal += (money_offered-card_offered.card_cost) / card_offered.card_cost * multiplier

            # check if the AI already owns a property in that color group
            card_group = card_offered.color_group
            num_properties_in_group = 0

            for card in ai.cards_owned:
                if card.color_group == card_group:
                    num_properties_in_group += 1

            benefit += 10 + (10 * num_properties_in_group)

        for card_name in properties_wanted:

            card_wanted = locate_card_object(card_name, board)

            # check if the AI already owns a property in that color group

            card_group = card_wanted.color_group
            num_properties_in_group = 0

            for card in ai.cards_owned:
                if card.color_group == card_group:
                    num_properties_in_group += 1

            benefit -= 40 + (num_properties_in_group * 10)

    # evaluate money exchange
    personal_gain += (money_offered - money_wanted) * multiplier/10

    # cost/benefit analysis:
    print(f"Personal gain: {personal_gain}. Human gain: {human_gain}")
    if personal_gain > human_gain + 10:
        return 1
    else:
        return 0



In [7]:
"""
@author: Aleph Aseffa
Date created: 7/14/2019

Contains the Player class and associated functions.
"""
import random



class Player:
    def __init__(self, name, balance, cards_owned, current_pos, in_jail, railroads_owned, doubles_counter, amount_owed, bankruptcy_status):
        self.name = name                            # str
        self.balance = balance                      # int
        self.cards_owned = cards_owned              # list
        self.current_pos = current_pos              # int (index)
        self.in_jail = in_jail                      # bool
        self.railroads_owned = railroads_owned      # int
        self.doubles_counter = doubles_counter      # int
        self.amount_owed = amount_owed              # int
        self.bankruptcy_status = bankruptcy_status  # bool

    def roll_dice(self):  # TODO: add check for doubles.
        """
        Simulates the randomness of throwing two die.
        :return: n: an int between 2 and 12 inclusive.
        """
        random.seed()
        dice1 = random.randint(1, 6)
        dice2 = random.randint(1, 6)
        n = dice1 + dice2
        print(f"{self.name} threw {n}")
        return n

    def move_player(self, dice_amt):
        """
        Moves the player by the amount returned by rolling two die.
        :param dice_amt: int, the number rolled by the two die.
        :return: an int that represents the updated position of the player on the board.
        """
        self.current_pos += dice_amt
        return self.current_pos

    def check_pos(self, board):
        """
        Checks what card the player has landed on and carries out the appropriate action.
        :param board: list, the monopoly board.
        :return: None
        """
        self.current_pos = self.current_pos % 40
        brd_property = board[self.current_pos]

        if brd_property.card_cost == "N/A":  # this means the player cannot purchase the card

            if brd_property.card_name == 'Jail/Visiting Jail':
                print(f"{self.name} is visiting jail.")

            elif brd_property.card_name == 'Luxury Tax':
                print(f"{self.name} landed on Luxury Tax and has been fined $75")
                self.reduce_balance(75)

            elif brd_property.card_name == 'Income Tax':
                print(f"{self.name} landed on Income Tax and has been fined $200")
                self.reduce_balance(200)

            elif brd_property.card_name == 'Go to Jail':
                print(f"{self.name} landed on Go to Jail and has been arrested!")
                self.send_to_jail()

            else:
                print(f"{self.name} landed on {brd_property.card_name}")

        else:
            if brd_property.mortgaged:
                print(f"{self.name} landed on a mortgaged property.")

            elif brd_property.owner != 'Bank':  # and brd_property.owner.name != self.name:
                if brd_property.owner.name == self.name:
                    print(f"{self.name} landed on {brd_property.card_name}, a property they own.")
                else:
                    print(f"{self.name} landed on {brd_property.card_name}, a property owned by {brd_property.owner.name}")
                    self.charge_rent(brd_property)

            else:
                print(f"{self.name} landed on {brd_property.card_name}")
                if self.name == "AI":
                    return 1  # Indicates to the AI that the property can be purchased
                else:
                    user_action = input("Do you want to buy the property? (y/n) ")
                    if user_action == 'y':
                        brd_property.purchase_card(self)

    def add_balance(self, amount):
        """
        Increases the player's balance.
        :param amount: int, the amount of money to add to the player's balance.
        :return: self.balance: the updated balance of the player.
        """
        self.balance += amount
        return self.balance

    def charge_rent(self, card):
        """
        Charges the rent cost to the player.
        :param card: an instance of the Card class.
        :return: None.
        """
        if card.color_group == "Railroad":
            rent_amt = 25 * card.owner.railroads_owned
        else:
            rent_amt = card.rent_prices[1]
        print(f"{self.name} is paying ${rent_amt} as a rental charge to {card.owner.name}")
        self.reduce_balance(rent_amt)
        card.owner.add_balance(rent_amt)

    def reduce_balance(self, amount):
        """
        Reduces the player's balance.
        :param amount: int, the amount of money to reduce the player's balance by.
        :return: None.
        """
        if self.balance < amount:
            print("Your balance is insufficient for this action.")
            bankrupt = self.check_if_bankrupt(amount)
            if not bankrupt:
                print("You need to sell or mortgage certain properties.")
                user_action = input("Do you want to sell or mortgage? (s/m)")
                if user_action == 's':
                    pass  # sell()  TODO: implement this function.
                else:
                    pass  # mortgage()  TODO: implement this function.
        else:
            self.balance -= amount

    def bankrupt_player(self):
        """
        Bank collects all the player's owned properties and sets their bankruptcy status to True.
        :return: None.
        """
        self.balance = 0

        if len(self.cards_owned):
            for card in self.cards_owned:
                card.owner = "Bank"
        self.railroads_owned = 0

        self.bankruptcy_status = True

    def check_if_bankrupt(self, amt_owed):
        """
        Checks if the player is bankrupt (i.e. can the player afford what they are being charged?).
        :param amt_owed: int, the amount the player is being charged.
        :return: Bool that represents if the player is bankrupt or not.
        """
        net_worth = 0

        for card in self.cards_owned:
            if card.mortgaged:
                net_worth -= card.mortgage_amt
                net_worth += card.card_cost
            else:
                net_worth += card.card_cost

        if (self.balance + net_worth) < amt_owed:
            print(f"Unfortunately, {self.name} is now bankrupt! It's game over for them!")
            self.bankrupt_player()
            return True
        else:
            return False

    def display_player_properties(self):
        """
        Prints out all the cards the player owns.
        :return: None.
        """
        total = 0
        for card in self.cards_owned:
            print(f"{card.card_name}: ${card.card_cost}")
            total += card.card_cost
        print(f"The sum of your card costs is: ${total}")

    def player_action(self, user_choice, player_list, computer, board):
        """
        Takes in the user's choice of what action to take and carries out that action.
        :param user_choice: char, what the player chooses to do (e.g. r to roll the dice).
        :return: None.
        """
        # TODO: add sell, mortgage, and construct house functions.
        val = -1
        if user_choice == "r":
            val = self.roll_dice()
        elif user_choice == "b":
            print(f"Your balance is: ${self.balance}")
        elif user_choice == "c":
            print("Your properties are:")
            self.display_player_properties()
        elif user_choice == "s":
            print("Sell property feature coming soon.")
        elif user_choice == "m":
            print("Mortgage property feature coming soon.")
        elif user_choice == "h":
            print("Construct house feature coming soon.")
        elif user_choice == "t":
            self.trade_with_ai(computer, board)
        elif user_choice == "p":
            trading_partner = input("Enter the name of the player you're trading with. ")
            self.trade_with_human(trading_partner, player_list, board)
        else:
            print("Please enter a valid command.")

        return val

    def send_to_jail(self):
        """
        Sends the player to jail.
        :return: None.
        """
        self.current_pos = 10
        self.doubles_counter = 0

    def release_from_jail(self):
        """
        Releases the player from jail.
        :return: None.
        """
        self.doubles_counter = 0
        bail_choice = input("Would you like to pay the $50 bail? (y/n) ")
        if bail_choice == "y":
            self.reduce_balance(50)
            self.in_jail = False
            dice_result = self.roll_dice()
            self.move_player(dice_result)
        else:
            self.roll_dice()
            if self.doubles_counter == 1:
                self.doubles_counter = 0
                dice_result = self.roll_dice()
                self.move_player(dice_result)

    def trade_with_human(self, player_reference, player_list, board):

        for player in player_list:
            if player.name == player_reference:
                other_player = player

        cash_given = int(input("How much cash are you giving away? "))
        properties_to_offer = input("Enter the properties do you want to offer separated by commas\n").split(',')

        cash_received = int(input(f"How much cash is {other_player.name} giving you?"))
        properties_received = input(f"Which properties is {other_player.name} giving you?\n").split(',')

        if properties_to_offer[0] == '':
            pass
        else:
            for card in properties_to_offer:
                card_object = locate_card_object(card, board)
                other_player.cards_owned.append(card_object)

        self.reduce_balance(cash_given)
        other_player.add_balance(cash_given)

        if properties_received[0] == '':
            pass
        else:
            for card in properties_received:
                card_object = locate_card_object(card, board)
                self.cards_owned.append(card_object)

        other_player.reduce_balance(cash_received)
        self.add_balance(cash_received)

        print(f"{self.name} has given ${cash_given} and the following properties: {properties_to_offer}")
        print(f"{other_player.name} has received ${cash_received} and the following properties: {properties_received}")

    def trade_with_ai(self, computer, board):
        cash_offered = int(input("How much cash do you want to offer? "))
        properties_to_offer = input("Enter the properties do you want to offer separated by commas\n").split(',')

        cash_wanted = int(input("How much cash do you want the AI to give you? "))
        properties_to_receive = input("Enter the properties you want the AI to give you (separated by commas)\n").split(',')

        if ai.evaluate_trade(self, computer, cash_offered, properties_to_offer, cash_wanted, properties_to_receive, board):
            print(f"The AI has accepted your trade offer and the trade has been completed.")

        else:
            retry = input("The AI has rejected your trade offer. Do you want to suggest a different trade? (y/n")
            if retry == "y":
                self.trade_with_ai(computer, board)


In [2]:
"""
@author: Aleph Aseffa
Date created: 7/14/2019

This file contains all the base information needed for the game to work.
This includes:
    - Board details
    - Community chest information
    - Chance information
"""

# card_name, color_group, card_cost, house_cost, houses_built, rent_prices, mortgage_amt, owner, mortgaged


def initialize_cards_and_board():
    """
    Creates all the needed card objects and adds them in the correct order to the board variable.
    The board represents the order of cards in the original Monopoly game board.
    :return: board: a list of cards.
    """
    go = Card("Go", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "Bank", False)
    med_ave = Card("Mediterranean Avenue", "Brown", 60, 50, 0, {0: 2,
                                                                      1: 10,
                                                                      2: 30,
                                                                      3: 90,
                                                                      4: 160,
                                                                      5: 250}, 30, "Bank", False)
    comm_chest = Card("Community Chest", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "Bank", False)
    baltic_ave = Card("Baltic Avenue", "Brown", 60, 50, 0, {0: 4,
                                                                  1: 20,
                                                                  2: 60,
                                                                  3: 180,
                                                                  4: 320,
                                                                  5: 450}, 30, "Bank", False)
    income_tax = Card("Income Tax", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "Bank", False)
    reading_rr = Card("Reading Railroad", "Railroad", 200, "N/A", "N/A", "N/A", 100, "Bank", False)
    oriental_ave = Card("Oriental Avenue", "Light Blue", 100, 50, 0, {0: 6,
                                                                            1: 30,
                                                                            2: 90,
                                                                            3: 270,
                                                                            4: 400,
                                                                            5: 550}, 50, "Bank", False)
    chance = Card("Community Chest", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "Bank", False)
    vermont_ave = Card("Vermont Avenue", "Light Blue", 100, 50, 0, {0: 6,
                                                                          1: 30,
                                                                          2: 90,
                                                                          3: 270,
                                                                          4: 400,
                                                                          5: 550}, 50, "Bank", False)
    conn_ave = Card("Connecticut Avenue", "Light Blue", 120, 50, 0, {0: 8,
                                                                           1: 40,
                                                                           2: 100,
                                                                           3: 300,
                                                                           4: 450,
                                                                           5: 600}, 60, "Bank", False)
    # new row
    jail = Card("Jail/Visiting Jail", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "Bank", False)
    st_charles_place = Card("St. Charles Place", "Pink", 140, 100, 0, {0: 10,
                                                                             1: 50,
                                                                             2: 150,
                                                                             3: 450,
                                                                             4: 625,
                                                                             5: 750}, 70, "Bank", False)
    electric_company = Card("Electric Company", "Utilities", 150, "N/A", "N/A", "N/A", 75, "Bank", False)
    states_ave = Card("States Avenue", "Pink", 140, 100, 0, {0: 10,
                                                                   1: 50,
                                                                   2: 150,
                                                                   3: 450,
                                                                   4: 625,
                                                                   5: 750}, 70, "Bank", False)
    virginia_ave = Card("Virginia Avenue", "Pink", 160, 100, 0, {0: 12,
                                                                       1: 60,
                                                                       2: 180,
                                                                       3: 500,
                                                                       4: 700,
                                                                       5: 900}, 80, "Bank", False)
    penn_rr = Card("Pennsylvania Railroad", "Railroad", 200, "N/A", "N/A", "N/A", 100, "Bank", False)
    st_james_place = Card("St. James Place", "Orange", 180, 100, 0, {0: 14,
                                                                           1: 70,
                                                                           2: 200,
                                                                           3: 550,
                                                                           4: 750,
                                                                           5: 950}, 90, "Bank", False)
    # comm chest
    ten_ave = Card("Tennessee Avenue", "Orange", 180, 100, 0, {0: 14,
                                                                     1: 70,
                                                                     2: 200,
                                                                     3: 550,
                                                                     4: 750,
                                                                     5: 950}, 90, "Bank", False)
    ny_ave = Card("New York Avenue", "Orange", 200, 100, 0, {0: 16,
                                                                   1: 80,
                                                                   2: 220,
                                                                   3: 600,
                                                                   4: 800,
                                                                   5: 1000}, 100, "Bank", False)
    free_parking = Card("Free Parking", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "Bank", False)
    # rotate
    kentucky_ave = Card("Kentucky Avenue", "Red", 220, 150, 0, {0: 18,
                                                                      1: 90,
                                                                      2: 250,
                                                                      3: 700,
                                                                      4: 875,
                                                                      5: 1050}, 110, "Bank", False)
    # chance
    indiana_ave = Card("Indiana Avenue", "Red", 220, 150, 0, {0: 18,
                                                                    1: 90,
                                                                    2: 250,
                                                                    3: 700,
                                                                    4: 875,
                                                                    5: 1050}, 110, "Bank", False)
    illinois_ave = Card("Illinois Avenue", "Red", 240, 150, 0, {0: 20,
                                                                      1: 100,
                                                                      2: 300,
                                                                      3: 750,
                                                                      4: 925,
                                                                      5: 1100}, 120, "Bank", False)
    bno_rr = Card("B. & O. Railroad", "Railroad", 200, "N/A", "N/A", "N/A", 100, "Bank", False)
    atlantic_ave = Card("Atlantic Avenue", "Yellow", 260, 150, 0, {0: 22,
                                                                         1: 110,
                                                                         2: 330,
                                                                         3: 800,
                                                                         4: 975,
                                                                         5: 1150}, 130, "Bank", False)
    ventnor_ave = Card("Ventnor Avenue", "Yellow", 260, 150, 0, {0: 22,
                                                                       1: 110,
                                                                       2: 330,
                                                                       3: 800,
                                                                       4: 975,
                                                                       5: 1150}, 130, "Bank", False)
    water_works = Card("Water Works", "Utilities", 150, 0, 0, {1: 100}, 75, "Bank", False)
    marvin_gardens = Card("Marvin Gardens", "Yellow", 280, 150, 0, {0: 24,
                                                                          1: 120,
                                                                          2: 360,
                                                                          3: 850,
                                                                          4: 1025,
                                                                          5: 1200}, 140, "Bank", False)
    # rotate
    go_to_jail = Card("Go to Jail", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "Bank", False)
    pacific_ave = Card("Pacific Avenue", "Green", 300, 200, 0, {0: 26,
                                                                      1: 130,
                                                                      2: 390,
                                                                      3: 900,
                                                                      4: 1100,
                                                                      5: 1275}, 150, "Bank", False)
    nc_ave = Card("North Carolina Avenue", "Green", 140, 200, 0, {0: 26,
                                                                        1: 130,
                                                                        2: 390,
                                                                        3: 900,
                                                                        4: 1100,
                                                                        5: 1275}, 150, "Bank", False)
    # comm chest
    penn_ave = Card("Pennsylvania Avenue", "Green", 300, 200, 0, {0: 28,
                                                                        1: 150,
                                                                        2: 450,
                                                                        3: 1000,
                                                                        4: 1200,
                                                                        5: 1400}, 160, "Bank", False)
    short_line_rr = Card("Short Line", "Railroad", 200, "N/A", "N/A", "N/A", 100, "Bank", False)
    # chance
    park_place = Card("Park Place", "Blue", 350, 200, 0, {0: 35,
                                                                1: 175,
                                                                2: 500,
                                                                3: 1100,
                                                                4: 1300,
                                                                5: 1500}, 175, "Bank", False)
    luxury_tax = Card("Luxury Tax", "N/A", "N/A", "N/A", "N/A", "N/A", 0, "Bank", False)
    boardwalk = Card("Boardwalk", "N/A", 400, 200, 0, {0: 50,
                                                             1: 200,
                                                             2: 600,
                                                             3: 1400,
                                                             4: 1700,
                                                             5: 2000}, 200, "Bank", False)
    # end
    board = [
        go,
        med_ave,
        comm_chest,
        baltic_ave,
        income_tax,
        reading_rr,
        oriental_ave,
        chance,
        vermont_ave,
        conn_ave,
        jail,
        st_charles_place,
        electric_company,
        states_ave,
        virginia_ave,
        penn_rr,
        st_james_place,
        comm_chest,
        ten_ave,
        ny_ave,
        free_parking,
        kentucky_ave,
        chance,
        indiana_ave,
        illinois_ave,
        bno_rr,
        atlantic_ave,
        ventnor_ave,
        water_works,
        marvin_gardens,
        go_to_jail,
        pacific_ave,
        nc_ave,
        comm_chest,
        penn_ave,
        short_line_rr,
        chance,
        park_place,
        luxury_tax,
        boardwalk
    ]

    return board


def display_instructions() -> None:
    """
    Displays possible options for players.
    :return: None
    """
    print("Instruction.......................................Command")
    print("Roll dice...........................................r")
    print("View balance........................................b")
    print("View cards and houses owned.........................c")
    print("Sell property.......................................s")
    print("Mortgage property...................................m")
    print("Construct house.....................................h")
    print("Trade with AI.......................................t")
    print("Trade with another player...........................p")
    print("View the AI's stats.................................v")


In [ ]:
"""
@author: Aleph Aseffa
Date created: 7/14/2019



"""

n1, n2 = username_form()

board = initialize_cards_and_board()

player1 = Player(n1, 1500, [], 0, False, 0, 0, 0, False)
player2 = Player(n2, 1500, [], 0, False, 0, 0, 0, False)
Computer = Player("AI", 1500, [], 0, False, 0, 0, 0, False)
player_list = [player1, player2, Computer]


def count_bankrupt_players(players):
    """
    Counts how many players have been bankrupted.
    :param players: list, players that are playing the game.
    :return: int, number of players that are bankrupt.
    """
    counter = 0
    for player in players:
        if player.bankruptcy_status:
            counter += 1
    return counter


def display_winner(players):
    """
    Prints out which player has won.
    :param players: list, players that are playing the game.
    :return: None.
    """
    for player in players:
        if player.bankruptcy_status == False:
            return player.name


if __name__ == "__main__":
    print("Beginning game!")
    display_instructions()

    i = 0
    # continue running while there is more than one non-bankrupt player remaining.
    
    while count_bankrupt_players(player_list) != len(player_list)-1:
        i = i % len(player_list)

        print()
        print(f"{player_list[i].name}'s turn:")

        if player_list[i].name == "AI":
            ai.move(Computer, board)

        else:
            user_choice = input("What do you want to do? ")
            result = player_list[i].player_action(user_choice, player_list, Computer, board)
            while result == -1:  # keep asking the user until they choose to roll the dice.
                user_choice = input("What do you want to do? ")
                result = player_list[i].player_action(user_choice, player_list, Computer, board)
            new_pos = player_list[i].move_player(result)
            player_list[i].check_pos(board)

        i += 1

    print(f"Game over! {display_winner(player_list)} has won!")

Player 1, please enter a username:Ryan
Player 2, please enter a username:Lucy
Beginning game!
Instruction.......................................Command
Roll dice...........................................r
View balance........................................b
View cards and houses owned.........................c
Sell property.......................................s
Mortgage property...................................m
Construct house.....................................h
Trade with AI.......................................t
Trade with another player...........................p
View the AI's stats.................................v

Ryan's turn:
